# Prize-Collecting Steiner Tree (PCSTP)

## Libs Importing

In [ ]:
import sys
import os
import time
import networkx as nx
from typing import Tuple, Set

sys.path.insert(1, os.path.realpath(os.path.pardir))

In [ ]:
from pcstp.instances.generator import generate_random_steiner
from pcstp.utils.draw import draw_steiner_graph

## Experiments

In [ ]:
G, (nodes, edges, position_matrix, edges_cost, terminals, prizes) = generate_random_steiner(
    num_nodes=25,
    num_edges=20,
    max_node_degree=10,
    min_prize=0,
    max_prize=100,
    num_terminals=5,
    min_edge_cost=0,
    max_edge_cost=10,
    cost_as_length=False,
    max_iter=100,
    seed=11
)

In [ ]:
from pcstp.steinertree import SteinerTreeProblem
from pcstp.instances.reader import SteinlibReader, DatReader
from pcstp.solver.base import computes_steiner_cost

The instance can be imported from a file or generated through the instance generator presented above.

In [ ]:
stp_reader = SteinlibReader()
stp = stp_reader.parser(filename='../data/instances/stp/C01-A.stp')

In [ ]:
stp = SteinerTreeProblem(graph=G, terminals=terminals)

In [ ]:
from pcstp.solver.base import computes_steiner_cost

In [ ]:
def preprocessing(graph: nx.Graph, terminals: set) -> Tuple[nx.Graph, Set[int]]:
    """Pre-Processing Function applied to Instace Graph in order to reduce complexity.

    It removes nodes with degree less or equal that are not terminal nodes

    Args:
        graph (nx.Graph): Instance graph
        terminals (set): Set of terminal nodes

    Returns:
        Tuple[nx.Graph, Set[int]]: Returns a processed version of Instance Graph and its terminals
    """
    final_graph = graph.copy()
    nodes_to_remove = [
        int(node) for node, degree in dict(graph.degree()).items()
        if degree <= 1 and node not in terminals 
    ]
    print(f'Removing nodes: {nodes_to_remove}')
    final_graph.remove_nodes_from(nodes_to_remove)

    return final_graph, terminals

In [ ]:
G, terminals = preprocessing(stp.graph, stp.terminals)

In [ ]:
stp_preprocessed = SteinerTreeProblem(graph=G, terminals=terminals)

## Solution obtained with NetworkX Steiner Tree Approximation Algorithm

In [ ]:
start_time = time.time()

nx_steiner_tree = nx.algorithms.approximation.steinertree.steiner_tree(
    stp_preprocessed.graph,
    stp_preprocessed.terminals,
    weight='cost'
)

networkx_duration = time.time() - start_time
networkx_cost = computes_steiner_cost(G, nx_steiner_tree, terminals)
print(f'Cost: {networkx_cost}')


In [ ]:
draw_steiner_graph(
    stp.graph,
    steiner_graph=nx_steiner_tree,
    plot_title=f'NetworkX Implementation - Cost ({networkx_cost}) - Time ({networkx_duration * 1000} ms)',
    node_label='name'
)


## Solution obtained with Greedy Heuristic Algorithm

In [ ]:
from pcstp.solver.greedy_h1 import GreedyH1

In [ ]:
# %%timeit -n 100

solver = GreedyH1(stp.graph, stp.terminals, log_level='info')
steiner_tree, greedy_cost = solver.solve()


In [ ]:
print(f'Cost: {solver.steiner_cost}')

In [ ]:
# %%timeit -n 100

preprocessed_graph_solver = GreedyH1(stp_preprocessed.graph, stp_preprocessed.terminals, log_level='info')
preprocessed_graph_steiner_tree, preprocessed_graph_greedy_cost = preprocessed_graph_solver.solve()

In [ ]:
print(f'Cost: {preprocessed_graph_greedy_cost}')

In [ ]:
draw_steiner_graph(
    stp.graph,
    steiner_graph=steiner_tree,
    plot_title=f'Greedy Heuristic (Shortest Path) - Cost ({greedy_cost}) - Time ({solver._duration * 1000} ms)',
    node_label='name'
)

In [ ]:
draw_steiner_graph(
    stp.graph,
    steiner_graph=preprocessed_graph_steiner_tree,
    plot_title=f'Greedy Heuristic (Shortest Path) - Cost ({preprocessed_graph_greedy_cost}) - Time ({preprocessed_graph_solver._duration * 1000} ms)',
    node_label='name'
)


In [ ]:
nodes_df, edges_df = stp.to_csv(path='../exports/instances/test')

In [ ]:
stp_2 = SteinerTreeProblem()
stp_2.read_csv('../exports/instances/test')

In [ ]:
stp_2.terminals